In [113]:
import numpy as np
import pandas as pd
import pickle
import bz2
import gzip
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
animelist = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")

In [3]:
animelist

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [4]:
rating

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [5]:
animelist.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [6]:
animelist.loc[((animelist["genre"]=="Hentai") | (animelist["type"]=="OVA") |(animelist["type"] == "Movie")) & (animelist["episodes"]=="Unknown"),"episodes"] = "1"

In [7]:
known_episodes = {"Naruto: Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin-chan":942, "Yu☆Gi☆Oh! Arc-V ":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25, "Dimension W: W no Tobira Online":5,
                  "Tanaka-kun wa Itsumo Kedaruge Specials":12, "Doraemon (2005)": 558, "Aggressive Retsuko":100, 
                 "Chibi Maruko-chan (1995)": 142, "Gudetama":10}
for k,v in known_episodes.items():    
    animelist.loc[animelist["name"]==k,"episodes"] = v

In [8]:
animelist["episodes"] = animelist["episodes"].map(lambda x:np.nan if x=="Unknown" else x)
animelist["episodes"].fillna(animelist["episodes"].median(),inplace = True)

In [9]:
animelist[animelist["rating"].isna()]

,anime_id,name,genre,type,episodes,rating,members
8968,34502,Inazma Delivery,"Action, Comedy, Sci-Fi",TV,10,NaN,32
9657,34309,Nananin no Ayakashi: Chimi Chimi Mouryou!! Gen...,"Comedy, Supernatural",TV,2.0,NaN,129
10896,34096,Gintama (2017),"Action, Comedy, Historical, Parody, Samurai, S...",TV,2.0,NaN,13383
10897,34134,One Punch Man 2,"Action, Comedy, Parody, Sci-Fi, Seinen, Super ...",TV,2.0,NaN,90706
10898,30484,Steins;Gate 0,"Sci-Fi, Thriller",NaN,2.0,NaN,60999
...,...,...,...,...,...,...,...
12274,34492,Nuki Doki! Tenshi to Akuma no Sakusei Battle -...,Hentai,OVA,1,NaN,392
12279,34491,Sagurare Otome The Animation,Hentai,OVA,1,NaN,79
12280,34312,Saimin Class,Hentai,OVA,1,NaN,240
12282,34388,Shikkoku no Shaga The Animation,Hentai,OVA,1,NaN,195


In [10]:
mean_rating = animelist["rating"].mean()
animelist["rating"].fillna(mean_rating, inplace=True)


In [11]:
animelist["genre"].fillna("anime", inplace=True)

In [12]:
animelist[animelist["type"].isna()]

,anime_id,name,genre,type,episodes,rating,members
10898,30484,Steins;Gate 0,"Sci-Fi, Thriller",NaN,2.0,6.473902,60999
10900,34437,Code Geass: Fukkatsu no Lelouch,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",NaN,2.0,6.473902,22748
10906,33352,Violet Evergarden,"Drama, Fantasy",NaN,2.0,6.473902,20564
10907,33248,K: Seven Stories,"Action, Drama, Super Power, Supernatural",NaN,2.0,6.473902,22133
10918,33845,Free! (Shinsaku),"School, Sports",NaN,2.0,6.473902,8666
10949,33475,Busou Shoujo Machiavellianism,"Action, School, Shounen",NaN,2.0,6.473902,1896
10963,31456,Code:Realize: Sousei no Himegimi,"Adventure, Fantasy, Romance, Shoujo",NaN,2.0,6.473902,4017
10983,34332,Flying Babies,anime,NaN,2.0,6.473902,22
10988,34280,Gamers!,"Comedy, Romance, School",NaN,2.0,6.473902,1045
10990,34485,Ganko-chan,anime,NaN,2.0,6.473902,11


In [13]:
known_types = {"Steins;Gate 0":"TV", "Code Geass: Fukkatsu no Lelouch": "Movie", "Violet Evergarden":"TV", "K: Seven Stories": "OVA",
              "Free! (Shinsaku)":"ONA", "Busou Shoujo Machiavellianism":"TV", "Code:Realize: Sousei no Himegimi":"TV", "Flying Babies":"TV",
              "Gamers!":"TV", "Ganko-chan":"TV","Gekidol":"TV","Ginga Eiyuu Densetsu (2017)":"OVA","Mirai Arise":"Movie","Ginga Jinpuu Jinraiger":"TV",
               "Grancrest Senki":"TV", "IDOLiSH7":"TV", "Isekai Shokudou":"TV", "Oushitsu Kyoushi Haine":"TV", "Peace Maker Kurogane (Shinsaku)":"ONA",
              "Rescue Academia":"TV", "Seikaisuru Kado":"TV", "UQ Holder!":"TV", "Citrus":"TV", "Hitorijime My Hero":"ONA", "Project758":"TV"}
for k,v in known_types.items():    
    animelist.loc[animelist["name"]==k,"type"] = v

In [14]:
animelist.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [15]:
animeWithName = pd.read_csv("MAL-anime.csv")
animeWithName = animeWithName.rename(columns={'Title': 'name'})

In [16]:
animeWithName = animelist.merge(animeWithName, on='name', how='left')
columns_to_drop = ['Unnamed: 0', 'Rank', 'Type', 'Episodes', 'Aired', 'Members', 'Score']
animeWithName = animeWithName.drop(columns=columns_to_drop)
animeWithName["image_url"].fillna('https://cdn.myanimelist.net/s/common/uploaded_files/1448416762-c4fea080c937bdde8ac476e4bfa76efe.png', inplace=True)
animeWithName.head()

,anime_id,name,genre,type,episodes,rating,members,page_url,image_url
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,https://myanimelist.net/anime/32281/Kimi_no_Na_wa,https://cdn.myanimelist.net/r/100x140/images/a...
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/r/100x140/images/a...
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,https://myanimelist.net/anime/28977/Gintama°,https://cdn.myanimelist.net/r/100x140/images/a...
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,https://myanimelist.net/anime/9253/Steins_Gate,https://cdn.myanimelist.net/r/100x140/images/a...
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,NaN,https://cdn.myanimelist.net/s/common/uploaded_...


In [17]:
animeWithName.isna().sum()

anime_id        0
name            0
genre           0
type            0
episodes        0
rating          0
members         0
page_url     4575
image_url       0
dtype: int64

In [18]:
rating.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

# popularity based recommender 

## Popularity Score = (Weighted Rating * Weight) + (Weighted Members * (1 - Weight))

In [19]:
popular_anime = animeWithName.copy()

In [20]:
weight = 0.65
max_rating = popular_anime["rating"].max()
max_members = popular_anime["members"].max()
popular_anime["weighted_rating"] = (popular_anime["rating"] / max_rating) * 10
popular_anime["weighted_members"] = (popular_anime["members"] / max_members) * 10

In [21]:
# Calculating the popularity score
popular_anime["popularity_score"] = (popular_anime["weighted_rating"] * weight) + (popular_anime["weighted_members"] * (1 - weight))

In [22]:
popular_anime["popularity_score"]

0        6.783067
1        8.758699
2        6.406928
3        8.285643
4        6.476164
           ...   
12289    2.698228
12290    2.782632
12291    3.172756
12292    3.237604
12293    3.549490
Name: popularity_score, Length: 12294, dtype: float64

In [23]:
# Sort anime by popularity score in descending order
most_popular_anime = popular_anime.sort_values(by="popularity_score", ascending=False)

In [24]:
print(most_popular_anime[["name", "popularity_score"]].head())

                                name  popularity_score
40                        Death Note          9.161500
1   Fullmetal Alchemist: Brotherhood          8.758699
86                Shingeki no Kyojin          8.644746
3                        Steins;Gate          8.285643
19   Code Geass: Hangyaku no Lelouch          8.208172


In [25]:
target_genre = "Fantasy"

In [26]:
genre_anime = most_popular_anime[most_popular_anime["genre"].str.contains(target_genre, case=False, na=False)].sort_values(by="popularity_score", ascending=False)
genre_anime.head(10)

,anime_id,name,genre,type,episodes,rating,members,page_url,image_url,weighted_rating,weighted_members,popularity_score
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/r/100x140/images/a...,9.26,7.827712,8.758699
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229,https://myanimelist.net/anime/16498/Shingeki_n...,https://cdn.myanimelist.net/r/100x140/images/a...,8.54,8.839274,8.644746
804,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100,https://myanimelist.net/anime/11757/Sword_Art_...,https://cdn.myanimelist.net/r/100x140/images/a...,7.83,8.808413,8.172445
118,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291,https://myanimelist.net/anime/19815/No_Game_No...,https://cdn.myanimelist.net/r/100x140/images/a...,8.47,5.940240,7.584584
200,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384,https://myanimelist.net/anime/121/Fullmetal_Al...,https://cdn.myanimelist.net/r/100x140/images/a...,8.33,5.921431,7.487001
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749,https://myanimelist.net/anime/4181/Clannad__Af...,https://cdn.myanimelist.net/r/100x140/images/a...,9.06,4.504797,7.465679
288,6702,Fairy Tail,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",TV,175,8.22,584590,https://myanimelist.net/anime/6702/Fairy_Tail,https://cdn.myanimelist.net/r/100x140/images/a...,8.22,5.765659,7.360981
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,784,8.58,504862,https://myanimelist.net/anime/21/One_Piece,https://cdn.myanimelist.net/r/100x140/images/a...,8.58,4.979323,7.319763
440,3588,Soul Eater,"Action, Adventure, Comedy, Fantasy, Shounen, S...",TV,51,8.08,580184,https://myanimelist.net/anime/3588/Soul_Eater,https://cdn.myanimelist.net/r/100x140/images/a...,8.08,5.722204,7.254771
643,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823,https://myanimelist.net/anime/9919/Ao_no_Exorcist,https://cdn.myanimelist.net/r/100x140/images/a...,7.92,5.758095,7.163333


# collabrative 


In [27]:
merged_df = rating.merge(animeWithName, on='anime_id', how='left')
merged_df.dropna(subset=['rating_x', 'name'], inplace=True)
merged_df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members,page_url,image_url
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0,https://myanimelist.net/anime/20/Naruto,https://cdn.myanimelist.net/r/100x140/images/a...
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553.0,https://myanimelist.net/anime/24/School_Rumble,https://cdn.myanimelist.net/r/100x140/images/a...
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772.0,https://myanimelist.net/anime/79/Shuffle,https://cdn.myanimelist.net/r/100x140/images/a...
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511.0,https://myanimelist.net/anime/226/Elfen_Lied,https://cdn.myanimelist.net/r/100x140/images/a...
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395.0,https://myanimelist.net/anime/241/Girls_Bravo_...,https://cdn.myanimelist.net/r/100x140/images/a...
...,...,...,...,...,...,...,...,...,...,...,...
7813732,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266.0,https://myanimelist.net/anime/16512/Devil_Surv...,https://cdn.myanimelist.net/r/100x140/images/a...
7813733,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747.0,NaN,https://cdn.myanimelist.net/s/common/uploaded_...
7813734,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895.0,https://myanimelist.net/anime/22145/Kuroshitsu...,https://cdn.myanimelist.net/r/100x140/images/a...
7813735,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005.0,https://myanimelist.net/anime/790/Ergo_Proxy,https://cdn.myanimelist.net/r/100x140/images/a...


In [75]:
# Count the number of ratings given by each user
user_rating_counts = merged_df['user_id'].value_counts().mean()
user_rating_counts

91.05228448275862

In [32]:
merged_df.loc[merged_df['rating_x'] == -1] = np.NaN
merged_df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members,page_url,image_url
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7813732,73515.0,16512.0,7.0,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266.0,https://myanimelist.net/anime/16512/Devil_Surv...,https://cdn.myanimelist.net/r/100x140/images/a...
7813733,73515.0,17187.0,9.0,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747.0,NaN,https://cdn.myanimelist.net/s/common/uploaded_...
7813734,73515.0,22145.0,10.0,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895.0,https://myanimelist.net/anime/22145/Kuroshitsu...,https://cdn.myanimelist.net/r/100x140/images/a...
7813735,73516.0,790.0,9.0,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005.0,https://myanimelist.net/anime/790/Ergo_Proxy,https://cdn.myanimelist.net/r/100x140/images/a...


In [115]:
# consider only expert users have have rated atleast 200 animes
x = merged_df.groupby('user_id').count()['rating_x'] > 200
experts = x[x].index

In [116]:
filtered_rating = merged_df[merged_df['user_id'].isin(experts)]

In [128]:
#consider anime rated by atleast 120 experts
y = filtered_rating.groupby('name').count()['rating_x'] > 160
famous_anime = y[y].index

In [129]:
final_ratings = filtered_rating[filtered_rating['name'].isin(famous_anime)]

In [130]:
final_ratings.drop_duplicates()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members,page_url,image_url
302,5.0,6.0,8.0,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069.0,https://myanimelist.net/anime/6/Trigun,https://cdn.myanimelist.net/r/100x140/images/a...
303,5.0,15.0,6.0,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648.0,https://myanimelist.net/anime/15/Eyeshield_21,https://cdn.myanimelist.net/r/100x140/images/a...
304,5.0,17.0,6.0,Hungry Heart: Wild Striker,"Comedy, Shounen, Slice of Life, Sports",TV,52,7.74,13469.0,https://myanimelist.net/anime/17/Hungry_Heart_...,https://cdn.myanimelist.net/r/100x140/images/a...
305,5.0,18.0,6.0,Initial D Fourth Stage,"Action, Cars, Drama, Seinen, Sports",TV,24,8.24,41584.0,https://myanimelist.net/anime/18/Initial_D_Fou...,https://cdn.myanimelist.net/r/100x140/images/a...
306,5.0,20.0,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0,https://myanimelist.net/anime/20/Naruto,https://cdn.myanimelist.net/r/100x140/images/a...
...,...,...,...,...,...,...,...,...,...,...,...
7813330,73507.0,8231.0,5.0,Princess Lover!: Magical Knight Maria-chan,"Magic, Parody",Special,6,5.87,5960.0,https://myanimelist.net/anime/8231/Princess_Lo...,https://cdn.myanimelist.net/r/100x140/images/a...
7813331,73507.0,8348.0,5.0,Tengen Toppa Gurren Lagann: Parallel Works 2,"Mecha, Music",Music,7,7.09,13361.0,NaN,https://cdn.myanimelist.net/s/common/uploaded_...
7813332,73507.0,8440.0,7.0,Black Lagoon Omake,Comedy,Special,7,7.02,33923.0,https://myanimelist.net/anime/8440/Black_Lagoo...,https://cdn.myanimelist.net/r/100x140/images/a...
7813333,73507.0,8769.0,8.0,Ore no Imouto ga Konnani Kawaii Wake ga Nai,"Comedy, Seinen, Slice of Life",TV,12,7.49,321477.0,https://myanimelist.net/anime/8769/Ore_no_Imou...,https://cdn.myanimelist.net/r/100x140/images/a...


In [131]:
pivot_table = final_ratings.pivot_table(index='name', columns = 'user_id', values='rating_x')
pivot_table.fillna(0, inplace=True)

In [132]:
pivot_table

user_id,5.0,7.0,17.0,38.0,43.0,46.0,123.0,129.0,139.0,160.0,...,73406.0,73417.0,73422.0,73457.0,73460.0,73476.0,73499.0,73502.0,73503.0,73507.0
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
.hack//G.U. Returner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//G.U. Trilogy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC,2.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,...,10.0,7.0,0.0,0.0,0.0,0.0,0.0,10.0,8.0,10.0
xxxHOLiC Kei,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,8.0,9.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,9.0


In [133]:
cos_sim_score = cosine_similarity(pivot_table)

In collaborative recommendation systems, cosine_similarity is used to calculate the similarity between users or items based on their ratings or interactions.

In [134]:
cos_sim_score.shape

(3056, 3056)

In [136]:
def get_recommendations(animeName):
#     find index of the anime in the pivot_table
    index = np.where(pivot_table.index==animeName)[0][0]
    
#     find the 10 most simmilar animes in cosine_similarity_table
    similar_anime_index = sorted(list(enumerate(cos_sim_score[index])),key=lambda x:x[1], reverse=True)[1:11]
    
#  get names of the similar animes
    similar_anime_list = []

    for i in similar_anime_index:
        similar_anime = animeWithName[animeWithName['name'] == pivot_table.index[i[0]]]
        similar_anime_list.append(similar_anime)
    similar_anime_df = pd.concat(similar_anime_list, ignore_index=True)
    return similar_anime_df

In [137]:
get_recommendations('Naruto')

,anime_id,name,genre,type,episodes,rating,members,page_url,image_url
0,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917,https://myanimelist.net/anime/1535/Death_Note,https://cdn.myanimelist.net/r/100x140/images/a...
1,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100,https://myanimelist.net/anime/11757/Sword_Art_...,https://cdn.myanimelist.net/r/100x140/images/a...
2,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151,https://myanimelist.net/anime/1575/Code_Geass_...,https://cdn.myanimelist.net/r/100x140/images/a...
3,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/r/100x140/images/a...
4,269,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055,https://myanimelist.net/anime/269/Bleach,https://cdn.myanimelist.net/r/100x140/images/a...
5,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229,https://myanimelist.net/anime/16498/Shingeki_n...,https://cdn.myanimelist.net/r/100x140/images/a...
6,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823,https://myanimelist.net/anime/9919/Ao_no_Exorcist,https://cdn.myanimelist.net/r/100x140/images/a...
7,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384,https://myanimelist.net/anime/121/Fullmetal_Al...,https://cdn.myanimelist.net/r/100x140/images/a...
8,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888,https://myanimelist.net/anime/2904/Code_Geass_...,https://cdn.myanimelist.net/r/100x140/images/a...
9,8074,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892,https://myanimelist.net/anime/8074/Highschool_...,https://cdn.myanimelist.net/r/100x140/images/a...


In [138]:
get_recommendations('Noragami')

,anime_id,name,genre,type,episodes,rating,members,page_url,image_url
0,30503,Noragami Aragoto,"Action, Adventure, Shounen, Supernatural",TV,13,8.48,299434,https://myanimelist.net/anime/30503/Noragami_A...,https://cdn.myanimelist.net/r/100x140/images/a...
1,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291,https://myanimelist.net/anime/19815/No_Game_No...,https://cdn.myanimelist.net/r/100x140/images/a...
2,22319,Tokyo Ghoul,"Action, Drama, Horror, Mystery, Psychological,...",TV,12,8.07,618056,https://myanimelist.net/anime/22319/Tokyo_Ghoul,https://cdn.myanimelist.net/r/100x140/images/a...
3,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229,https://myanimelist.net/anime/16498/Shingeki_n...,https://cdn.myanimelist.net/r/100x140/images/a...
4,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100,https://myanimelist.net/anime/11757/Sword_Art_...,https://cdn.myanimelist.net/r/100x140/images/a...
5,15809,Hataraku Maou-sama!,"Comedy, Demons, Fantasy, Romance, Shounen",TV,13,8.03,409037,https://myanimelist.net/anime/15809/Hataraku_M...,https://cdn.myanimelist.net/r/100x140/images/a...
6,18153,Kyoukai no Kanata,"Fantasy, Slice of Life, Supernatural",TV,12,7.88,359011,https://myanimelist.net/anime/18153/Kyoukai_no...,https://cdn.myanimelist.net/r/100x140/images/a...
7,20787,Black Bullet,"Action, Mystery, Sci-Fi, Seinen",TV,13,7.44,320267,https://myanimelist.net/anime/20787/Black_Bullet,https://cdn.myanimelist.net/r/100x140/images/a...
8,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823,https://myanimelist.net/anime/9919/Ao_no_Exorcist,https://cdn.myanimelist.net/r/100x140/images/a...
9,18897,Nisekoi,"Comedy, Harem, Romance, School, Shounen",TV,20,7.91,347599,https://myanimelist.net/anime/18897/Nisekoi,https://cdn.myanimelist.net/r/100x140/images/a...


In [139]:
pickle.dump(most_popular_anime, open('most_popular.pkl','wb'))

In [140]:
# pickle.dump(pivot_table, open('pivot_table.pkl','wb'))
# pickle.dump(cos_sim_score, open('cos_sim_score.pkl','wb'))
pickle.dump(animeWithName, open('animeWithName.pkl','wb'))

In [141]:
with gzip.open('pivot_table2.gz', 'wb') as file:
    pickle.dump(pivot_table, file)

In [142]:
with gzip.open('cos_sim_score2.gz', 'wb') as file:
    pickle.dump(cos_sim_score, file)